This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/anomaly_detection).

## House Price Testing Saga

This tutorial simulates using Wallaroo assays to detect model or data drift, and Wallaroo Shadow Deploy to compare different models to determine which one is most fit for an organization's needs.  These two features allow organizations to monitor model performance and accuracy, then swap them out as needed.

* **IMPORTANT NOTE**: This tutorial assumes that historical data is available for the assays functionality.  The code for creating and using assays has been commented out, but is made available for examination and as examples.

This tutorial will demonstrate how to:

1. Select or create a workspace, pipeline and upload the champion model.
1. Add a pipeline step with the champion model, then deploy the pipeline and perform sample inferences.
1. Create an assay baseline and display 
1. Establish a pipeline and add the model as a step.
1. Create an assay with baseline data and display anomalies.
1. Swap out the pipeline step with the champion model with a shadow deploy step that compares the champion model against two competitors.
1. Evaluate the results of the champion versus competitor models.
1. Swap out the shadow deploy pipeline step with the new challenger model.
1. Undeploy the pipeline.

This tutorial provides the following:

* Models:
  * `rf_model.onnx`: The champion model that has been used in this environment for some time.
  * `xgb_model.onnx` and `models/gbr_model.onnx`: Rival models that will be tested against the champion.
* Data:
  * xtest-1.df.json and xtest-1k.df.json:  DataFrame JSON inference inputs with 1 input and 1,000 inputs.
  * xtest-1.arrow and xtest-1k.arrow:  Apache Arrow inference inputs with 1 input and 1,000 inputs.

## Prerequisites

* A deployed Wallaroo instance
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * [`polars`](https://pypi.org/project/polars/): Polars for DataFrame with native Apache Arrow support

## Steps

### Import libraries

The first step is to import the libraries needed for this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import json

from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import polars

import os
# For Wallaroo SDK 2023.1
os.environ["ARROW_ENABLED"]="True"

### Connect to Wallaroo Instance

The following command will create a connection to the Wallaroo instance and store it in the variable `wl`.

In [16]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR PREFIX"

wallarooPrefix = "doc-test"
wallarooSuffix = "wallaroocommunity.ninja"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

In [17]:
workspace_name = 'houseprice'
main_pipeline_name = 'housepricepipeline'
model_name_control = 'housingcontrol'
model_file_name_control = './models/rf_model.onnx'
model_name_challenger_01 = 'housingchallenger01'
model_name_challenger_02 = 'housingchallenger02'

In [18]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [19]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'houseprice', 'id': 12, 'archived': False, 'created_by': 'ce15ed10-9217-48fb-a9bf-ecbff480a87a', 'created_at': '2023-04-04T18:12:23.229104+00:00', 'models': [], 'pipelines': []}

## Upload The Models

For our example, we will upload three models, all pre-trained to determine housing prices based on various variables.

The assumption is that we have a pipeline deployed that has been determining house prices for some time using 

* `rf_model.onnx`: The champion model that has been used in this environment for some time.
* `xgb_model.onnx` and `models/gbr_model.onnx`: Rival models that will be tested against the champion.

We will upload our control model into our workspace, then deploy a pipeline with the `rf_model.onnx` and perform some sample inferences.

In [20]:
housing_model_control = wl.upload_model(model_name_control, model_file_name_control).configure()

### Build the Control Sample Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.  We'll call it `housepricepipeline`, set `rf_model.onnx` as a pipeline step as set in the variable declarations above, and run a few sample inferences.

In [21]:
mainpipeline = wl.build_pipeline(main_pipeline_name).add_model_step(housing_model_control).deploy()

### Testing

We'll use two inferences as a quick sample test - one that has a house that should be determined around $700k, the other with a house determined to be around $1.5 million.

In [9]:
# normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
# result = mainpipeline.infer(normal_input)
# display(result)

In [10]:
# large_house_input = pd.DataFrame.from_records({'tensor': [[4.0, 3.0, 3710.0, 20000.0, 2.0, 0.0, 2.0, 5.0, 10.0, 2760.0, 950.0, 47.6696, -122.261, 3970.0, 20000.0, 79.0, 0.0, 0.0]]})
# large_house_result = mainpipeline.infer(large_house_input)
# display(large_house_result)

As one last sample, we'll run through roughly 1,000 inferences at once and show a few of the results.

In [22]:
large_inference_result = pd.DataFrame()

for x in range(100):
    large_inference_result = large_inference_result.append(mainpipeline.infer_from_file("./data/xtest-1k.df.json"))

In [23]:
display(large_inference_result)

,time,in.tensor,out.variable,check_failures
0,2023-04-04 18:12:51.375,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-04-04 18:12:51.375,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-04-04 18:12:51.375,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-04-04 18:12:51.375,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-04-04 18:12:51.375,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
...,...,...,...,...
995,2023-04-04 18:15:08.357,"[3.0, 2.5, 2900.0, 23550.0, 1.0, 0.0, 0.0, 3.0, 10.0, 1490.0, 1410.0, 47.5708, -122.153, 2900.0, 19604.0, 27.0, 0.0, 0.0]",[827411.0],0
996,2023-04-04 18:15:08.357,"[4.0, 1.75, 2700.0, 7875.0, 1.5, 0.0, 0.0, 4.0, 8.0, 2700.0, 0.0, 47.454, -122.144, 2220.0, 7875.0, 46.0, 0.0, 0.0]",[441960.38],0
997,2023-04-04 18:15:08.357,"[4.0, 3.25, 2910.0, 1880.0, 2.0, 0.0, 3.0, 5.0, 9.0, 1830.0, 1080.0, 47.616, -122.282, 3100.0, 8200.0, 100.0, 0.0, 0.0]",[1060847.5],0
998,2023-04-04 18:15:08.357,"[3.0, 1.75, 2910.0, 37461.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1530.0, 1380.0, 47.7015, -122.164, 2520.0, 18295.0, 47.0, 0.0, 0.0]",[706823.56],0


In [25]:
topic = wl.get_topic_name(mainpipeline.id())

from datetime import datetime

past_inferences = wl.get_raw_pipeline_inference_logs(topic=topic, start=datetime.fromtimestamp(0), end=datetime.now(), model_name=model_name_control)

In [26]:
len(past_inferences)

100000

In [30]:
display(past_inferences.head(5))

,time,in,out,check_failures,metadata
0,1680631971375,"{'tensor': [4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]}",{'variable': [718013.75]},[],"{'last_model': '{""model_name"":""housingcontrol"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}'}"
1,1680631971375,"{'tensor': [2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]}",{'variable': [615094.56]},[],"{'last_model': '{""model_name"":""housingcontrol"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}'}"
2,1680631971375,"{'tensor': [3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]}",{'variable': [448627.72]},[],"{'last_model': '{""model_name"":""housingcontrol"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}'}"
3,1680631971375,"{'tensor': [4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]}",{'variable': [758714.2]},[],"{'last_model': '{""model_name"":""housingcontrol"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}'}"
4,1680631971375,"{'tensor': [3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]}",{'variable': [513264.7]},[],"{'last_model': '{""model_name"":""housingcontrol"",""model_sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}'}"


In [28]:
import joblib

# pickle them
with open('houseprice_logs.pickle', 'wb') as f:
    joblib.dump(past_inferences, f)

In [29]:
mainpipeline.undeploy()

name,housepricepipeline
created,2023-04-04 18:12:27.091404+00:00
last_updated,2023-04-04 18:12:27.866858+00:00
deployed,False
tags,
versions,"0413ebb2-96cd-4a6e-a8ea-3fa9cc387834, 9d810ca8-35c8-4b3c-9496-7e277fc87fa3"
steps,housingcontrol


In [12]:
loghistory = mainpipeline.logs(limit=100000000)

In [13]:
display(loghistory)

,time,in.tensor,out.variable,check_failures
0,2023-04-04 14:41:16.297,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-04-04 14:41:16.297,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-04-04 14:41:16.297,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-04-04 14:41:16.297,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-04-04 14:41:16.297,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0
...,...,...,...,...
379,2023-04-04 14:41:16.297,"[5.0, 2.5, 1630.0, 20750.0, 1.0, 0.0, 0.0, 4.0, 7.0, 1100.0, 530.0, 47.3657, -122.113, 1630.0, 8640.0, 40.0, 0.0, 0.0]",[278094.94],0
380,2023-04-04 14:41:16.297,"[5.0, 1.75, 2320.0, 7700.0, 1.0, 0.0, 0.0, 5.0, 7.0, 1290.0, 1030.0, 47.3426, -122.285, 1740.0, 7210.0, 52.0, 0.0, 0.0]",[347191.72],0
381,2023-04-04 14:41:16.297,"[2.0, 2.5, 1900.0, 5065.0, 2.0, 0.0, 0.0, 3.0, 8.0, 1900.0, 0.0, 47.7175, -122.034, 1350.0, 4664.0, 10.0, 0.0, 0.0]",[441512.56],0
382,2023-04-04 14:41:16.297,"[3.0, 1.75, 1020.0, 7020.0, 1.5, 0.0, 0.0, 4.0, 7.0, 1020.0, 0.0, 47.7362, -122.314, 1020.0, 5871.0, 61.0, 0.0, 0.0]",[340764.53],0


## Shadow Deploy

The other method for comparing models is Shadow Deploy.  In Shadow Deploy, the pipeline step is added with the `add_shadow_deploy` method, with the champion model listed first, then an array of challenger models after.  **All** inference data is fed to **all** models, with the champion results displayed in the `out.variable` column, and the shadow results in the format `out_{model name}.variable`.  For example, since we named our challenger models `housingchallenger01` and `housingchallenger02`, the columns `out_housingchallenger01.variable` and `out_housingchallenger02.variable` have the shadow deployed model results.

Here, we'll create a new pipeline called `housepriceshadowtesting`, then add `rf_model.onnx` as our champion, and models `xgb_model.onnx` and `gbr_model.onnx` as the challengers.  We'll deploy the pipeline and prepare it for sample inferences.

In [90]:
shadow_pipeline_name = 'housepriceshadowtesting'
shadow_pipeline = wl.build_pipeline(shadow_pipeline_name).add_shadow_deploy(housing_model_control, [housing_model_challenger01, housing_model_challenger02]).deploy()

### Shadow Deploy Sample Inference

We'll now use our same sample data for an inference to our shadow deployed pipeline.

In [91]:
shadow_result = shadow_pipeline.infer_from_file('./data/xtest-1.df.json')

display(shadow_result)

,time,in.tensor,out.variable,check_failures,out_housingchallenger01.variable,out_housingchallenger02.variable
0,2023-03-29 20:27:05.104,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0,[659806.0],[704901.9]


### Shadow Deploy Batch Inference

We can also perform batch inferences with shadow deployed pipelines.  Here we'll pass 1,000 inference requests at once, then display the results.

In [92]:
shadow_results = shadow_pipeline.infer_from_file('./data/xtest-1k.arrow')

outputs =  shadow_results.to_pandas()

display(outputs.loc[:,['out.variable','out_housingchallenger01.variable','out_housingchallenger02.variable']])

,out.variable,out_housingchallenger01.variable,out_housingchallenger02.variable
0,[718013.75],[659806.0],[704901.9]
1,[615094.56],[732883.5],[695994.44]
2,[448627.72],[419508.84],[416164.8]
3,[758714.2],[634028.8],[655277.2]
4,[513264.7],[427209.44],[426854.66]
...,...,...,...
995,[827411.0],[743487.94],[787589.25]
996,[441960.38],[381577.16],[411258.3]
997,[1060847.5],[1520770.0],[1491293.8]
998,[706823.56],[663008.75],[594914.2]


### Shadow Deploy Logs

Shadow deployed results are also displayed in the log files.  For Arrow enabled Wallaroo instances, it's just the pipeline `logs` method.  For Arrow disabled environments, the command `logs_shadow_deploy()` displays the shadow deployed model information.

In [93]:
logs = shadow_pipeline.logs()
display(logs.loc[:,['out.variable','out_housingchallenger01.variable','out_housingchallenger02.variable']])

,out.variable,out_housingchallenger01.variable,out_housingchallenger02.variable
0,[937281.75],[779282.94],[1074469.5]
1,[450996.25],[431373.0],[433386.56]
2,[266405.6],[308694.3],[277806.5]
3,[727923.1],[612911.25],[521321.47]
4,[480151.5],[712661.25],[605705.8]
...,...,...,...
95,[236238.66],[158358.28],[180559.94]
96,[581003.0],[651141.75],[647863.06]
97,[446768.88],[451848.2],[397378.5]
98,[400628.5],[392271.75],[368913.88]


### Undeploy Shadow Pipeline

We can now undeploy the shadow deployed pipeline to return the resources back to the Wallaroo instance.

In [94]:
shadow_pipeline.undeploy()

name,housepriceshadowtesting
created,2023-03-29 19:31:26.274598+00:00
last_updated,2023-03-29 20:26:53.952065+00:00
deployed,False
tags,
versions,"31dcab61-21ac-4dc6-8de8-755caf994edf, cbe5c049-6ad8-443c-b905-9420cf1f930e, d46ddeac-9bd0-4020-b111-8320445b5e7d, dec7c584-ce2f-44ed-af37-9cb3785f4929"
steps,housingcontrol


## Model Swap

Now that we've completed our testing, we can swap our deployed model in the original `housepricingpipeline` with one we feel works better.  This is one with the pipeline `replace_with_model_step` method, where we specify the pipeline step and the model to replace it with.  This pipeline had only one step with the `rf_model.onnx` model, and we'll swap it out with the `gbr_model.onnx` model.

The model swap capability makes updating a pipeline with new models a quick production process.  

We'll do an inference with the current model, then swap out the old for the new, then another inference check.

In [95]:
# inference before model swap

display(mainpipeline.status())
swapinference = mainpipeline.infer_from_file('./data/xtest-1.df.json')
display(swapinference)

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.2.19',
   'name': 'engine-646bff55b5-nj9gj',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'housepricepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'housingcontrol',
      'version': '87a75b3f-1cbc-46bb-9aca-066c42162bdc',
      'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.2.20',
   'name': 'engine-lb-ddd995646-p8b7j',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

,time,in.tensor,out.variable,check_failures
0,2023-03-29 20:27:48.941,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


In [96]:
# Swap the model with a new one, then redeploy the pipeline
mainpipeline.replace_with_model_step(0, housing_model_challenger01).deploy()

name,housepricepipeline
created,2023-03-29 17:27:50.527879+00:00
last_updated,2023-03-29 20:27:49.364409+00:00
deployed,True
tags,
versions,"00ad5db7-68f8-4c9a-b8d3-e11ecac82fe4, c65cbc00-e018-494f-83df-753fcf5fcd6b, 6140d071-f271-444c-a1b9-e2bbe8d00e92, 4859b778-52de-4c94-8230-4eaaf596ce32, 63c53886-9d38-44d0-a4df-a5ced0d2af98, 660af202-adff-4e23-966b-2b20e4138663, b0ffbd14-b58b-43f7-888d-37e0709cd90a, 6c9a282b-85fb-46e8-8e2e-3e804022b243, 2c1fe269-52d5-4288-9941-72cbbc3f8a45, 745e6ec4-9264-4f4b-8780-f9237cdd2569, 950430fd-2319-4e79-b3fe-986c50e219cf, 9a40b7d7-c65a-46fc-9b29-134fd65cf34b, 24de628e-c2b7-4a98-a81b-25c66d69f57a, d759b5b8-ee4d-408c-902a-022d6a773d0a, c3c67191-79b7-4690-a04c-0ee72eef2de5, 0245d76a-0221-496f-8e3f-9bff780a8dbe"
steps,housingcontrol


In [97]:
# inference after model swap

display(mainpipeline.status())
swapinference = mainpipeline.infer_from_file('./data/xtest-1.df.json')
display(swapinference)

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.2.19',
   'name': 'engine-646bff55b5-nj9gj',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'housepricepipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'housingchallenger01',
      'version': '419f15fa-4256-48f5-9a2c-9d2393f5f787',
      'sha': '31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.2.20',
   'name': 'engine-lb-ddd995646-p8b7j',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

,time,in.tensor,out.variable,check_failures
0,2023-03-29 20:27:58.480,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0


### Undeploy Main Pipeline

With the examples and tutorial complete, we will undeploy the main pipeline and return the resources back to the Wallaroo instance.

In [23]:
mainpipeline.undeploy()

name,housepricepipeline
created,2023-04-04 14:40:50.831417+00:00
last_updated,2023-04-04 14:40:51.547137+00:00
deployed,False
tags,
versions,"7406a914-2412-4d27-add4-ec0bb5f5d657, bb0ec99c-8c97-4b08-91e3-93dd81da13ae"
steps,housingcontrol
